In [1]:
import sys
import json
import os
from category_encoders.leave_one_out import LeaveOneOutEncoder
import numpy as np
import joblib
from catboost.utils import read_cd
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from catboost.utils import read_cd
from sklearn.metrics import zero_one_loss, log_loss, roc_auc_score, precision_recall_curve, auc
from scipy.stats import ttest_rel
import math
import joblib
from collections import defaultdict
import pandas as pd

In [2]:
def create_dir(name):
    directory = os.path.dirname(name)
    if not os.path.exists(name):
        os.makedirs(name)

In [2]:
def process_classification_dataset(name):
    
    # load and prepare data
    data_dir = os.path.join('datasets', name)
    train_file = os.path.join(data_dir, 'full_train')
    test_file = os.path.join(data_dir, 'test')
    cd_file = os.path.join(data_dir, 'pool.cd')

    train = np.loadtxt(train_file, delimiter="\t", dtype="object")
    test = np.loadtxt(test_file, delimiter="\t", dtype="object")
    cd = read_cd(cd_file, data_file=train_file)

    # Target can be called 'Label' or 'Target' in pool.cd
    try:
        label_ind = cd['column_type_to_indices']['Label']
    except:
        label_ind = cd['column_type_to_indices']['Target']

    np.random.seed(42)  # fix random seed
    train = np.random.permutation(train)

    y_train = train[:, label_ind]
    y_train = y_train.reshape(-1)

    y_test = test[:, label_ind]
    y_test = y_test.reshape(-1)
    
    try:
        cat_features = cd['column_type_to_indices']['Categ']  # features to be replaced
        enc = LeaveOneOutEncoder(cols=cat_features, return_df=False, random_state=10, sigma=0.3)
    except:
        enc = LeaveOneOutEncoder(return_df=False, random_state=10, sigma=0.3)

    transformed_train = enc.fit_transform(train, y_train).astype("float64")
    X_train = np.delete(transformed_train, label_ind, 1)  # remove target column

    transformed_test = enc.transform(test).astype("float64")
    X_test = np.delete(transformed_test, label_ind, 1)  # remove target column

    return np.nan_to_num(X_train), y_train, np.nan_to_num(X_test), y_test, enc

In [4]:
def generate_rf_ensemble_classification(dataset_name, num_models=10, 
                                        n_estimators = 1000, compress=3, 
                                        n_jobs=-1, max_depth=15):

    X_train, y_train, _, _, _ = process_classification_dataset(dataset_name)

    seed = 0
    for i in range(num_models):
    
        model = RandomForestClassifier(n_estimators=n_estimators,
                                       max_depth=max_depth, n_jobs=n_jobs, 
                                       random_state=seed, max_features='sqrt', criterion='entropy')

        seed += 1 # new seed for each ensemble element

        model.fit(X_train, y_train) 
        joblib.dump(model, "results/models/" + dataset_name + "_" + "rf" + "_" + str(i), compress=compress)

In [9]:
datasets = ["credit"]

create_dir("results/models")

print('Classification - Bagging: Generating models..')
for name in datasets:
    print("Dataset =", name)

    # Training all models
    print("Training models...")
    generate_rf_ensemble_classification(name)
print('Done!!')

Classification - Bagging: Generating models..
Dataset = credit
Training models...
Done!!


----

In [3]:
def prr_class(labels, probs, measure, rev: bool):
    # Get predictions
    preds = np.argmax(probs, axis=1)

    if rev:
        inds = np.argsort(measure)[::-1]
    else:
        inds = np.argsort(measure)

    total_data = np.float64(preds.shape[0])
    errors, percentages = [], []

    for i in range(preds.shape[0]):
        errors.append(np.sum(
            np.asarray(labels[inds[:i]] != preds[inds[:i]], dtype=np.float32)) * 100.0 / total_data)
        percentages.append(float(i + 1) / total_data * 100.0)
    errors, percentages = np.asarray(errors)[:, np.newaxis], np.asarray(percentages)

    base_error = errors[-1]
    n_items = errors.shape[0]
    
    auc_uns = 1.0 - auc(percentages / 100.0, errors[::-1] / 100.0)

    random_rejection = np.asarray(
        [base_error * (1.0 - float(i) / float(n_items)) for i in range(n_items)],
        dtype=np.float32)
    auc_rnd = 1.0 - auc(percentages / 100.0, random_rejection / 100.0)
    orc_rejection = np.asarray(
        [base_error * (1.0 - float(i) / float(base_error / 100.0 * n_items)) for i in
         range(int(base_error / 100.0 * n_items))], dtype=np.float32)
    orc = np.zeros_like(errors)
    orc[0:orc_rejection.shape[0]] = orc_rejection
    auc_orc = 1.0 - auc(percentages / 100.0, orc / 100.0)

    rejection_ratio = (auc_uns - auc_rnd) / (auc_orc - auc_rnd) * 100.0
    return rejection_ratio

def ood_detect(domain_labels, in_measure, out_measure, mode, pos_label=1):
    scores = np.concatenate((in_measure, out_measure), axis=0)
    scores = np.asarray(scores, dtype=np.longdouble)
    if pos_label != 1:
        scores *= -1.0

    if mode == 'PR':
        precision, recall, thresholds = precision_recall_curve(domain_labels, scores)
        aupr = auc(recall, precision)
        return aupr

    elif mode == 'ROC':
        roc_auc = roc_auc_score(domain_labels, scores)
        return roc_auc

def nll_class(target, probs, epsilon=1e-10):
    log_p = -np.log(probs + epsilon)
    return target*log_p[:, 1] + (1-target)*log_p[:, 0]

In [4]:
def entropy(probs, epsilon=1e-10):
    log_probs = -np.log(probs + epsilon)
    return np.sum(probs * log_probs, axis=1)

def entropy_of_expected_class(probs, epsilon=1e-10):
    mean_probs = np.mean(probs, axis=0)
    log_probs = -np.log(mean_probs + epsilon)
    return np.sum(mean_probs * log_probs, axis=1)


def expected_entropy_class(probs, epsilon=1e-10):
    log_probs = -np.log(probs + epsilon)

    return np.mean(np.sum(probs * log_probs, axis=2), axis=0)

In [5]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))
     
def load_model(name, alg, i):
    if alg == "rf":
        model = joblib.load("results/models/" + name + "_" + alg + "_" + str(i))
    else:
        model = CatBoostClassifier()
        model.load_model("results/models/" + name + "_" + alg + "_" + str(i)) 
    return model

def normalize_test_labels(y_test):
    y_test_norm = []
    c0 = min(y_test)
    for y in y_test:
        if y == c0:
            y_test_norm.append(0)
        else:
            y_test_norm.append(1)
    return np.array(y_test_norm)
            
def aggregate_results(name, modes = ["single", "ens"], 
                      algorithms = ["sglb-fixed", "rf"], num_models = 10):
    

    results = [] # metric values for all algorithms and all folds
        
    for mode in modes:
        for alg in algorithms:
        
            if alg == "rf":
                train_pool, y_train, test_pool, y_test, enc = process_classification_dataset(name)
                
                # process ood data
                cd = read_cd("datasets/"+name+"/pool.cd", data_file = "datasets/"+name+"/test")
                try: 
                    label_ind = cd['column_type_to_indices']['Label']
                except:
                    label_ind = cd['column_type_to_indices']['Target']

                ood_test_pool = np.loadtxt("datasets/ood/" + name + "_rf", delimiter="\t", dtype="object")
                ood_test_pool = enc.transform(ood_test_pool).astype("float64")
                ood_test_pool = np.delete(ood_test_pool, label_ind, 1)
                ood_size = len(ood_test_pool)
                
            else:
                test_pool = Pool(data="datasets/"+name+"/test", column_description="datasets/"+name+"/pool.cd")
                ood_test_pool = Pool(data="datasets/ood/" + name, column_description="datasets/"+name+"/pool.cd")
                ood_size = ood_test_pool.num_row()

                y_test = test_pool.get_label()
            
            test_size = len(y_test)
            domain_labels = np.concatenate([np.zeros(test_size), np.ones(ood_size)])
                    
            y_test_norm = normalize_test_labels(y_test)
        
            values = defaultdict() # metric values for all folds for given algorithm

            if mode == "single":
                # use 0th model from ensemble as a single model
                model = load_model(name, alg, 0)
                preds = model.predict(test_pool)
                preds_proba = model.predict_proba(test_pool)
    
                values["error"] = (preds != y_test).astype(int)
                values["nll"] = nll_class(y_test_norm, preds_proba)
                values["TU_prr"] = prr_class(y_test_norm, preds_proba, entropy(preds_proba), False)
                values["KU_prr"] = float("nan")
                values["KU_auc"] = float("nan")
                    
                ood_preds_proba = model.predict_proba(ood_test_pool)
                in_measure = entropy(preds_proba)
                out_measure = entropy(ood_preds_proba)
                values["TU_auc"] = ood_detect(domain_labels, in_measure, out_measure, mode="ROC")

            if mode == "ens":
                all_preds = [] # predictions of all models in ensemble
                all_preds_ood = []
                    
                for i in range(num_models):
                    model = load_model(name, alg, i)
                    preds = model.predict_proba(test_pool)
                    all_preds.append(preds)
                    preds = model.predict_proba(ood_test_pool)
                    all_preds_ood.append(preds) 
                        
                all_preds = np.array(all_preds)
                preds_proba = np.mean(all_preds, axis=0)
                
                all_preds_ood = np.array(all_preds_ood)
                
                preds = np.argmax(preds_proba, axis=1)
                values["error"] = (preds != y_test_norm).astype(int)
                values["nll"] = nll_class(y_test_norm, preds_proba)
                
                TU = entropy_of_expected_class(all_preds)
                DU = expected_entropy_class(all_preds)
                KU = TU - DU
                
                TU_ood = entropy_of_expected_class(all_preds_ood)
                DU_ood = expected_entropy_class(all_preds_ood)
                KU_ood = TU_ood - DU_ood

                values["TU_prr"] = prr_class(y_test_norm, preds_proba, TU, False)
                values["KU_prr"] = prr_class(y_test_norm, preds_proba, KU, False)
                  
                values["TU_auc"] = ood_detect(domain_labels, TU, TU_ood, mode="ROC")
                values["KU_auc"] = ood_detect(domain_labels, KU, KU_ood, mode="ROC")
                        
            results.append(values)

    return np.array(results)
    
def make_table(values):
    prr_TU = np.array([values[i]["TU_prr"] for i in range(len(values))])
    auc_TU = np.array([values[i]["TU_auc"] for i in range(len(values))])
    TU = np.concatenate((prr_TU, 100*auc_TU), axis=0)

    prr_KU = np.array([values[i]["KU_prr"] for i in range(len(values))])
    auc_KU = np.array([values[i]["KU_auc"] for i in range(len(values))])
    KU = np.concatenate((prr_KU, 100*auc_KU), axis=0)

    df = pd.DataFrame(np.stack((TU, KU)), index=['TU', 'KU']) 
    columns=[('PRR%', 'Single','SGLB'),('PRR%', 'Single','RF'), ('PRR%', 'Ensemble','SGLB'),('PRR%', 'Ensemble','RF'), 
             ('AUC-ROC%', 'Single','SGLB'),('AUC-ROC%', 'Single','RF'), ('AUC-ROC%', 'Ensemble','SGLB'),('AUC-ROC%', 'Ensemble','RF')]
    df.columns=pd.MultiIndex.from_tuples(columns)
    return df

In [6]:
dataset = "credit"

print('\t\t\tClassification - Bagging')
print("\t===Comparison with random forest, PRR and AUC-ROC===")

values = aggregate_results(dataset, algorithms=["sglb-fixed", "rf"], modes=["single", "ens"])

make_table(values)

			Classification - Bagging
	===Comparison with random forest, PRR and AUC-ROC===


PRR%                                   AUC-ROC%             \
       Single              Ensemble               Single              
         SGLB         RF       SGLB         RF      SGLB         RF   
TU  46.238564  55.082168  45.858059  55.584319  75.49434  77.019936   
KU        NaN        NaN  18.123800  29.850301       NaN        NaN   

                          
     Ensemble             
         SGLB         RF  
TU  75.308638  77.871898  
KU  99.484801  63.443752